In [ ]:
print('Setup complete.')

# Lab 03: Building a Custom MCP Server

## Learning Objectives
- Consolidate your understanding of the MCPS architecture
- Design and build a custom MCP server from scratch
- Wrap a custom tool (a simple calculator) as an MCP resource
- Implement an agent that uses your custom server to solve problems

## Setup

In [ ]:
from typing import List, Dict, Any, Callable
from dataclasses import dataclass, field
import operator

## Part 1: Defining the Custom Tool

First, we need a tool to wrap. We'll create a simple calculator that can perform basic arithmetic operations. This will be the core logic our MCP server exposes.

In [ ]:
def calculator(a: float, b: float, op: str) -> str:
    """Performs a basic arithmetic operation."""
    operations = {
        'add': operator.add,
        'subtract': operator.sub,
        'multiply': operator.mul,
        'divide': operator.truediv
    }
    
    if op not in operations:
        return f"Error: Invalid operation '{op}'. Must be one of {list(operations.keys())}."
    
    try:
        result = operations[op](a, b)
        return f'The result of {a} {op} {b} is {result}.'
    except ZeroDivisionError:
        return 'Error: Cannot divide by zero.'
    except Exception as e:
        return f'An error occurred: {e}'

## Part 2: Building the Custom MCP Server

Now, we'll build the MCP server that will make our `calculator` tool available to an AI agent. We will reuse the server infrastructure from the previous labs.

In [ ]:
# --- MCP Server Infrastructure (from Lab 01 & 02) ---
@dataclass
class MCPResource:
    name: str
    description: str
    handler: Callable[..., Any]

class CustomMCPServer:
    def __init__(self, name: str):
        self.name = name
        self.resources: Dict[str, MCPResource] = {}

    def register_resource(self, resource: MCPResource):
        self.resources[resource.name] = resource
        print(f'Registered resource "{resource.name}" on server "{self.name}".')

    def list_resources(self) -> List[Dict[str, str]]:
        return [{'name': r.name, 'description': r.description} for r in self.resources.values()]

    def execute_resource(self, name: str, args: Dict) -> Any:
        if name not in self.resources: return f'Error: Resource "{name}" not found.'
        try:
            return self.resources[name].handler(**args)
        except Exception as e: return f'Error executing resource: {e}'

# --- Create and configure our custom server ---

# 1. Instantiate the server
calculator_server = CustomMCPServer(name='calculator-mcp')

# 2. Define the resource based on our tool
calculator_resource = MCPResource(
    name='calculate',
    description='Performs an arithmetic calculation. Args: a (float), b (float), op (str: add, subtract, multiply, divide).',
    handler=calculator
)

# 3. Register the resource
calculator_server.register_resource(calculator_resource)

print("
Server setup complete! Available resources:")
print(calculator_server.list_resources())

## Part 3: Building an Agent to Use the Calculator Server

In [ ]:
class CalculatorAgent:
    def __init__(self, server: CustomMCPServer):
        self.server = server

    def solve_problem(self, problem: str) -> str:
        # A mock LLM that parses the problem and generates a tool call
        tool_call = self._parse_problem_to_tool_call(problem)
        
        if not tool_call:
            return "I can only solve simple arithmetic problems."
            
        print(f'Agent decided to call tool with args: {tool_call["args"]}')
        
        # Execute the tool call using the server
        result = self.server.execute_resource(tool_call['resource'], tool_call['args'])
        return result

    def _parse_problem_to_tool_call(self, problem: str) -> Dict:
        problem = problem.lower()
        # Simple regex-based parsing for this mock agent
        match = re.search(r'what is (\d+\.?\d*)\s*(\w+)\s*(\d+\.?\d*)', problem)
        if not match:
            return None
        
        a, op_str, b = match.groups()
        op_map = {'plus': 'add', 'minus': 'subtract', 'times': 'multiply', 'divided by': 'divide'}
        op = op_map.get(op_str)
        
        if not op:
            return None
            
        return {
            'server': 'calculator-mcp',
            'resource': 'calculate',
            'args': {'a': float(a), 'b': float(b), 'op': op}
        }

# --- Run the agent ---
agent = CalculatorAgent(server=calculator_server)

problem1 = 'What is 15 times 4?'
result1 = agent.solve_problem(problem1)
print(f'\nProblem: {problem1}')
print(f'Answer: {result1}')

problem2 = 'What is 100 divided by 5?'
result2 = agent.solve_problem(problem2)
print(f'\nProblem: {problem2}')
print(f'Answer: {result2}')

## Exercises

1. **Add a `power` Operation**: Add the ability for the `calculator` to handle exponentiation (e.g., `power` or `**`). Update the `operations` dictionary and the agent's parsing logic to handle a query like, "What is 2 to the power of 8?".
2. **Create a `constants` Resource**: Add a new resource to the server called `get_constant(name: str)` that returns mathematical constants like 'pi' (3.14159) or 'e' (2.71828). Update the agent to be able to answer questions like, "What is the value of pi?".
3. **Improve Agent's Robustness**: The agent's parsing logic is very fragile. Instead of using regex, try to write a prompt that would ask a real LLM to extract the numbers and operation, and return them as a JSON object. This would make the agent much more flexible.

## Summary

In this lab, you consolidated everything you've learned about MCPS:
- You defined a **custom tool** with its own logic (`calculator`).
- You built a **custom MCP server** from scratch to host your tool as a resource.
- You created an **agent** that can understand a user's problem, generate the correct tool call for your custom server, and use the result to provide an answer.
- This end-to-end process demonstrates the power and modularity of the MCPS architecture for building capable, tool-using AI agents.